In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from copy import deepcopy


In [2]:

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Class for Collaborative Filtering
class CollabDataset(Dataset):
    def __init__(self, df, user_col=0, item_col=1, rating_col=2):
        self.df = df
        self.user_tensor = torch.tensor(self.df.iloc[:, user_col], dtype=torch.long, device=device)
        self.item_tensor = torch.tensor(self.df.iloc[:, item_col], dtype=torch.long, device=device)
        self.target_tensor = torch.tensor(self.df.iloc[:, rating_col], dtype=torch.float32, device=device)
        
    def __getitem__(self, index):
        return (self.user_tensor[index], self.item_tensor[index], self.target_tensor[index])

    def __len__(self):
        return self.user_tensor.shape[0]



In [3]:
# Function to load data
def load_data(data_path, id_val=1):
    # Load train and validation data
    train_df = pd.read_csv(f'{data_path}u{id_val}.base', sep='\t', header=None)
    train_df.columns = ['user_id', 'item_id', 'rating', 'ts']
    train_df['user_id'] = train_df['user_id'] - 1
    train_df['item_id'] = train_df['item_id'] - 1

    valid_df = pd.read_csv(f'{data_path}u{id_val}.test', sep='\t', header=None)
    valid_df.columns = ['user_id', 'item_id', 'rating', 'ts']
    valid_df['user_id'] = valid_df['user_id'] - 1
    valid_df['item_id'] = valid_df['item_id'] - 1

    return train_df, valid_df



In [4]:
# Function to get DataLoaders
def get_data_loaders(train_df, valid_df, batch_size=2000):
    train_dataset = CollabDataset(train_df)
    valid_dataset = CollabDataset(valid_df)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    return train_loader, valid_loader



In [5]:
# Define the Neural Network for the Recommendation System
class ConcatNet(nn.Module):
    def __init__(self, config):
        super(ConcatNet, self).__init__()
        self.config = config
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.emb_size = config['emb_size']
        self.emb_dropout = config['emb_dropout']
        self.fc_layer_sizes = config['fc_layer_sizes']
        self.dropout = config['dropout']
        self.out_range = config['out_range']

        self.emb_user = nn.Sequential(
            nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.emb_size),
            nn.Dropout(p=self.emb_dropout))
        self.emb_item = nn.Sequential(
            nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.emb_size),
            nn.Dropout(p=self.emb_dropout))

        fc_layers_list = []
        for ni, nf, p in zip(self.fc_layer_sizes[:-1], self.fc_layer_sizes[1:], self.dropout):
            fc_layers_list.append(nn.Linear(ni, nf))
            fc_layers_list.append(nn.ReLU(inplace=True))
            fc_layers_list.append(nn.BatchNorm1d(nf))
            fc_layers_list.append(nn.Dropout(p=p))
        self.fc_layers = nn.Sequential(*fc_layers_list)

        self.head = torch.nn.Linear(in_features=self.fc_layer_sizes[-1], out_features=1)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.emb_user[0].weight)
        nn.init.xavier_uniform_(self.emb_item[0].weight)

    def forward(self, user_idx, item_idx):
        user_emb = self.emb_user(user_idx)
        item_emb = self.emb_item(item_idx)
        x = torch.cat([user_emb, item_emb], dim=1)
        x = self.fc_layers(x)
        x = torch.sigmoid(self.head(x))
        x = x * (self.out_range[1] - self.out_range[0]) + self.out_range[0]
        return x



In [6]:
# Configuration for the model
config = {
    'num_users': 943,
    'num_items': 1682,
    'emb_size': 50,
    'emb_dropout': 0.05,
    'fc_layer_sizes': [100, 512, 256],
    'dropout': [0.7, 0.35],
    'out_range': [0.8, 5.2]
}



In [7]:
# Training function
def train_model(data_path, num_epochs=5, batch_size=2000, learning_rate=1e-2, weight_decay=5e-1):
    train_df, valid_df = load_data(data_path)
    train_loader, valid_loader = get_data_loaders(train_df, valid_df, batch_size)

    model = ConcatNet(config).to(device)
    criterion = torch.nn.MSELoss(reduction='sum')  # Using sum to compute total loss
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

    best_loss = np.inf
    train_losses = []
    valid_losses = []
    valid_rmse = []
    valid_mae = []

    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0
        for u, i, r in train_loader:
            u, i, r = u.to(device), i.to(device), r.to(device)
            r_pred = model(u, i)
            loss = criterion(r_pred, r[:, None])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        valid_loss = 0
        total_abs_error = 0
        total_squared_error = 0
        num_samples = 0
        with torch.no_grad():
            for u, i, r in valid_loader:
                u, i, r = u.to(device), i.to(device), r.to(device)
                r_pred = model(u, i)
                valid_loss += criterion(r_pred, r[:, None]).item()
                
                # Accumulate MAE and RMSE
                abs_error = torch.abs(r_pred - r[:, None]).sum().item()
                squared_error = torch.sum((r_pred - r[:, None]) ** 2).item()
                
                total_abs_error += abs_error
                total_squared_error += squared_error
                num_samples += len(r)

        valid_loss /= len(valid_loader.dataset)
        valid_losses.append(valid_loss)

        # Calculate MAE and RMSE
        mae = total_abs_error / num_samples
        rmse = np.sqrt(total_squared_error / num_samples)

        valid_rmse.append(rmse)
        valid_mae.append(mae)

        scheduler.step(valid_loss)
        print(f"Epoch {epoch + 1}: Train Loss = {train_loss}, Valid Loss = {valid_loss}, RMSE = {rmse}, MAE = {mae}")

        if valid_loss < best_loss:
            best_loss = valid_loss
            best_model = deepcopy(model.state_dict())

    # Load the best model and save it
    model.load_state_dict(best_model)
    torch.save(model.state_dict(), 'best_model.pth')

    # Plot the losses and metrics
    plt.plot(train_losses, label='Train Loss')
    plt.plot(valid_losses, label='Valid Loss')
    plt.plot(valid_rmse, label='RMSE')
    plt.plot(valid_mae, label='MAE')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()
    plt.show()



In [8]:
# !pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
# !pip install --upgrade jupyter ipywidgets


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [9]:
# Call the train_model function
data_path = "E:\Projects\youtube-Recomendation system/notebooks/ml-100k/"  # Replace with the correct path to your data
train_model(data_path, num_epochs=10)


  0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
import os


os.getcwd()

'C:\\Users\\ASUS'